In [ ]:
import os
import pylab
import string
import fdsreader
import matplotlib
import subprocess

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from importlib import reload
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter
import spotpy
from scipy.stats.qmc import LatinHypercube
from pyDOE import lhs
import random
import shutil

## Load Experiment data

In [ ]:
sims_path = os.path.join("../BurnerSims/LHC_Cases_030423")
os.listdir(sims_path)

In [ ]:
seed_fld="cases0_6062"
sims_path_seed=os.path.join(sims_path,seed_fld)

In [ ]:
# main path
LHC_sample_csv=pd.read_csv(f'TableData/case0_param_values.csv', header=0)

# Path to experiment data
exp_root = os.path.join("../GeneralInformation/macfp-db")
exp_macfp_dir = os.path.join(exp_root+"/Fire_Growth/NIST_Parallel_Panel/Experimental_Data")

In [ ]:
# Load the table ofthe centerline - Burner_HF_Centerline_multi-layer.csv
centreline_hf_path = os.path.join(exp_macfp_dir, "Burner_HF_Centerline_multi-layer.csv")
centreline_hf_df = pd.read_csv(centreline_hf_path, header=0, skiprows=[1])

# Load the table ofthe centerline - Burner_steadyHF_Width_multi-layer.csv
ST_Width_layer_path = os.path.join(exp_macfp_dir, "Burner_steadyHF_Width_multi-layer.csv")
ST_Width_layer_df = pd.read_csv(ST_Width_layer_path, header=0, skiprows=[1])


In [ ]:
#Data of the experiment Center - time and Width - steady state
hf_TIME_center = centreline_hf_df[["HF_z20", "HF_z50", "HF_z75", "HF_z100"]].to_numpy()
hf_STEADY_width = ST_Width_layer_df[["HF_y-25", "HF_y-15", "HF_y0", "HF_y15", "HF_y25"]].to_numpy()

## Prepare Sim results

### SIM: Avg HF **CENTER**

In [ ]:
# Define DEVC IDs and desired times
devc_ids = ["HF_y0_z20", "HF_y0_z50", "HF_y0_z75", "HF_y0_z100"]
desired_times = [20, 40, 60, 80]  # s

# Define time window to average over
frame_window = 3

# Create an empty dictionary to store the results
SIM_avg_HF_Center_dict = {}

# Loop through each subfolder and read the CSV file
for subdir in os.listdir(sims_path_seed):
    if subdir.startswith("MaCFP_CASE0_sample"):
        # Extract the sample number from the subfolder name
        sample_num = subdir.split("_")[-1]
        
        # Find the DEVC CSV file in the subfolder
        for file in os.listdir(os.path.join(sims_path_seed, subdir)):
            if file.endswith("_devc.csv") and file.startswith("MaCFP_Burner_CASE0_sample") and file.endswith(f"{sample_num}_devc.csv"):
                # Read the DEVC data from the CSV file
                devc_data = pd.read_csv(os.path.join(sims_path_seed, subdir, file), header=1)

                # Create an empty 2D array to store average flux values and MSEs
                SIM_avg_Fluxes_center = np.zeros((len(devc_ids) + 1, len(desired_times)))

                # Compute the average flux values for each DEVC ID and desired time
                for i, devc_id in enumerate(devc_ids):
                    for j, desired_time in enumerate(desired_times):
                        # Compute time window to average over
                        frames = desired_time * 2
                        t_min = int(frames - frame_window)
                        t_max = int(frames + frame_window + 1)

                        # Compute average within the time window
                        flux_avrg = np.average(devc_data[devc_id][t_min:t_max].to_numpy())

                        # Store the average flux value in the array
                        SIM_avg_Fluxes_center[i, j] = flux_avrg

                # Compute the mean squared error (MSE) for each desired time
                for j, desired_time in enumerate(desired_times):
                    SIM_avg_Fluxes_center[-1, j] = np.mean((SIM_avg_Fluxes_center[:-1, j] - hf_TIME_center) ** 2)

                # Add the results to the dictionary using the sample number as the key
                SIM_avg_HF_Center_dict[f'Case0_{sample_num}'] = SIM_avg_Fluxes_center

# Print the number of samples
print(len(SIM_avg_HF_Center_dict))

# Convert the nested dictionary to a dataframe and save it to a CSV file
data = {key: value.flatten() for key, value in SIM_avg_HF_Center_dict.items()}
columns = []
for j, desired_time in enumerate(desired_times):
    for devc_id in devc_ids:
        columns.append(f"{devc_id}_T{desired_time}")
    columns.append(f"MSE_T{desired_time}")

df = pd.DataFrame.from_dict(data, orient="index", columns=columns)
df.index.name = "sample_num"
df.to_csv("TableData/SIM_avg_HF_Center.csv")


In [ ]:
pd.read_csv("TableData/SIM_avg_HF_Center.csv", header=0)

### SIM: **SURFACE** HF 

In [ ]:
# Define DEVC IDs using nested loops
heights = ["z20", "z50", "z75", "z100"]
y_positions = ["-25", "-15", "0", "15", "25"]
devc_ids = [f"HF_y{y_pos}_{height}" for height in heights for y_pos in y_positions]

# Function to process a single file
def process_devc_file(filepath):
    # Read the DEVC data from the CSV file
    devc_data = pd.read_csv(filepath, header=1)
    
    # Compute the average for each device column over the last 20 entries
    devc_data_avg = devc_data[devc_ids].tail(20).mean().to_dict()

    # Compute the mean squared error for the averaged data
    mse = ((np.array(list(devc_data_avg.values())) - hf_STEADY_width) ** 2).mean()

    # Add the mse to the devc_data_avg dictionary
    devc_data_avg["mse"] = mse

    return devc_data_avg

# Loop through each subfolder and read the CSV file
SIM_HF_surface = {}
for subdir in os.listdir(sims_path_seed):
    if subdir.startswith("MaCFP_CASE0_sample"):
        # Extract the sample number from the subfolder name
        sample_num = subdir.split("_")[-1]
        
        # Find the DEVC CSV file in the subfolder
        devc_csv_pattern = f"MaCFP_Burner_CASE0_sample{sample_num}_devc.csv"
        csv_files = [file for file in os.listdir(os.path.join(sims_path_seed, subdir)) if file.startswith(devc_csv_pattern)]
        
        if csv_files:
            # Process the file and add the results to the dictionary
            filepath = os.path.join(sims_path_seed, subdir, csv_files[0])
            SIM_HF_surface[sample_num] = process_devc_file(filepath)

# Convert the dictionary to a dataframe and save it to a CSV file
df = pd.DataFrame.from_dict(results, orient="index", columns=devc_ids+['MSE'])
df.index.name = "sample_num"
df.to_csv("TableData/SIM_HF_surface.csv")


In [ ]:
pd.read_csv("TableData/SIM_HF_surface.csv", header=0)

## LHC Parameter overview and distribution

In [ ]:
# Plot LHC Parameters for an overview
# Create a grid of plots with 2 columns
fig, axs = plt.subplots(nrows=len(data_csv.columns), ncols=2, figsize=(12, 24))

# Analyze distribution and plot line for each column
for i, col in enumerate(LHC_sample_csv.columns):
    axs[i, 0].hist(LHC_sample_csv[col],bins=20)
    axs[i, 0].set_title(f"Distribution of {col}")
    axs[i, 0].set_xlabel(col)
    axs[i, 0].set_ylabel("Frequency")
    axs[i, 1].plot(data_csv[col])
    axs[i, 1].set_title(col)
    axs[i, 1].set_xlabel("Index")
    axs[i, 1].set_ylabel(col)

plt.tight_layout()
plt.show()


# Compare parameter with result

## Surface

In [ ]:
# Read the CSV files
lhc_df = pd.read_csv("TableData/cases0_param_values.csv")
sim_hf_df = pd.read_csv("TableData/SIM_HF_surface.csv")

# Extract the MSE values 
mse_values = sim_hf_df.loc[:, sim_hf_df.columns.str.startswith("MSE")].values

# Loop through the columns in the LHC_sample_csv.csv file and create a plot for each parameter
for column in lhc_df.columns:
    plt.figure()
    
    # Plot the parameter values on the y-axis against the MSE values on the x-axis
    plt.scatter(mse_values, lhc_df[column])

    # Set plot labels and title
    plt.xlabel("MSE")
    plt.ylabel(column)
    plt.title(f"{column} vs. MSE")

 
    # Show the plot
    plt.show()


## Time 20 s

In [ ]:
# Read the CSV files
lhc_df = pd.read_csv("TableData/cases0_param_values.csv")
sim_hf_df = pd.read_csv("TableData/SIM_avg_HF_Center.csv")

# Extract the MSE values 
mse_values = sim_hf_df.loc[:, sim_hf_df.columns.str.startswith("MSE_T20")].values

# Loop through the columns in the LHC_sample_csv.csv file and create a plot for each parameter
for column in lhc_df.columns:
    plt.figure()
    
    # Plot the parameter values on the y-axis against the MSE values on the x-axis
    plt.scatter(mse_values, lhc_df[column])

    # Set plot labels and title
    plt.xlabel("MSE")
    plt.ylabel(column)
    plt.title(f"{column} vs. MSE")

 
    # Show the plot
    plt.show()
